In [1]:
import sys
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
# import preprocessing.preprocess
!pwd
# from legendary_game_recs.preprocessing import preprocess
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess import *

/Users/antonis/code/Ant-mel/legendary_game_recs/notebooks


In [128]:
raw_data = pd.read_csv('../raw_data/all_data_batch1_30to60k')
raw_2 = pd.read_csv('../raw_data/all_data_batch1_0to4k')
raw_3 = pd.read_csv('../raw_data/all_data_batch2_4kto8k')
raw_4 = pd.read_csv('../raw_data/all_data_batch3_8kto12k')
raw_5 = pd.read_csv('../raw_data/all_data_batch_60kto70k')
raw_6 = pd.read_csv('../raw_data/all_data_batch_70kto80k')
raw_7 = pd.read_csv('../raw_data/all_data_batch_80kto90k')

In [129]:
data = pd.concat((raw_2, raw_3, raw_4, raw_data, raw_5, raw_6), ignore_index=True)

In [149]:
list_of_str = ['Ltd.', 'salmon', 'chicken', "LLC", 'poop']

def remove_generic_company_abbreviations(list_):
    """This removes trailing company abberviations that were apperaing as individual companies"""
    words_list = ['Ltd.', 'Inc.', 'LLC'] 
    
    for word in words_list:
        if word in list_:
            list_.remove(word)
    
    return list_

def keep_x_OHE_columns(OHE_coulmns, num_features=10):
    """Slices by the X (num_featutes) most common columns"""
    """Returns only the amount specified"""
    cols_by_power = pd.DataFrame(OHE_coulmns.sum().sort_values(ascending=False))
    
    top_20_OHE = list(cols_by_power[0:num_features].index)
    cols_to_keep = OHE_coulmns[top_20_OHE]
    
    return cols_to_keep

In [134]:
categories_as_lists = make_list_columns_to_lists(data, ['developers', 'platforms', 'genres'])

In [161]:
list(categories_as_lists.columns)

['developers', 'platforms', 'genres']

In [176]:
def keeping_ohe_columns_and_dropping_the_originals(list_of_columns, num_ohe):
    empty = pd.DataFrame()
    
    columns = list(list_of_columns.columns)
    
    for col in columns:
        encoded_col, col_encoder = categorical_encoder(list_of_columns[col])
        cols_to_keep = keep_x_OHE_columns(encoded_col, num_ohe)
        empty = pd.concat((empty, cols_to_keep), axis=1)
        
    return empty

In [178]:
keeping_ohe_columns_and_dropping_the_originals(categories_as_lists, 5).columns

Index(['', 'Nintendo', 'Sega', 'Square Enix', 'Electronic Arts', 'Windows PC',
       'Mac', 'PlayStation 4', 'Nintendo Switch', 'Linux', 'Indie',
       'Adventure', 'RPG', 'Simulator', 'Strategy'],
      dtype='object')

# Developers

In [136]:
dev = categories_as_lists['developers'].apply(remove_generic_abbreviations)

encoded_dev, dev_encoder = categorical_encoder(dev)


In [145]:
devs_to_keep = keep_x_OHE_columns(encoded_dev, 5)

In [146]:
devs_to_keep

,,Nintendo,Sega,Square Enix,Electronic Arts
0,0,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
61913,0,0,0,0,0
61914,0,0,0,0,0
61915,1,0,0,0,0
61916,0,0,0,0,0


# Genres

In [100]:
genres = categories_as_lists['genres']

In [101]:
encoded_genre, genre_encoder = categorical_encoder(genres)

In [147]:
genre_by_power = encoded_genre.sum().sort_values(ascending=False)

In [148]:
genre_to_keep = keep_x_OHE_columns(encoded_genre, 5)
genre_to_keep

,Indie,Adventure,RPG,Simulator,Strategy
0,1,0,0,0,0
1,1,1,0,0,0
2,0,0,1,0,0
3,0,0,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
61913,0,1,0,0,0
61914,0,0,0,1,0
61915,0,0,0,0,0
61916,1,0,0,0,0


# Platforms

In [150]:
platforms = categories_as_lists['platforms']

encoded_plat, plat_encoder = categorical_encoder(platforms)

platforms_by_power = pd.DataFrame(encoded_plat.sum().sort_values(ascending=False))

In [152]:
platforms_to_keep = keep_x_OHE_columns(encoded_plat, 5)
platforms_to_keep

,Windows PC,Mac,PlayStation 4,Nintendo Switch,Linux
0,0,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
61913,1,1,0,0,0
61914,1,0,0,0,0
61915,0,0,0,0,0
61916,1,1,0,0,1


In [156]:
empty = pd.DataFrame()

In [171]:
empty.append(platforms_to_keep)

/var/folders/62/d7t21rj54ls65fhypjm4ryb00000gn/T/ipykernel_27637/3800765986.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empty.append(platforms_to_keep)


,Windows PC,Mac,PlayStation 4,Nintendo Switch,Linux
0,0,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
61913,1,1,0,0,0
61914,1,0,0,0,0
61915,0,0,0,0,0
61916,1,1,0,0,1


In [155]:
pd.concat((platforms_to_keep, devs_to_keep, genre_to_keep), axis=1)

,Windows PC,Mac,PlayStation 4,Nintendo Switch,Linux,,Nintendo,Sega,Square Enix,Electronic Arts,Indie,Adventure,RPG,Simulator,Strategy
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61913,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
61914,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
61915,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
61916,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0


In [ ]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []
    
    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)
    
    return remove_whitespace_df

In [ ]:
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()
    
    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df

In [ ]:
updated_df = make_list_columns_to_lists(data, ['genres', 'platforms', 'developers'])

In [ ]:
updated_df

In [ ]:
mlb_genre = MultiLabelBinarizer()
mlb_platform = MultiLabelBinarizer()
mlb_developer = MultiLabelBinarizer()


transfomed_genre = mlb_genre.fit_transform(updated_df['genres'])
transformed_platform = mlb_platform.fit_transform(updated_df['platform'])
transfomed_developer = mlb_developer.fit_transform(updated_df['developer'])

In [ ]:
genre_encoded = pd.DataFrame(transfomed, columns=mlb.classes_)

In [ ]:
def categorical_encoder(category):
    mlb = MultiLabelBinarizer()
    transformed = mlb.fit_transform(category)
    
    df = pd.DataFrame(transformed, columns=mlb.classes_)
    
    return df, mlb


In [ ]:
df, mlb_thing = categorical_encoder(updated_df['developers'])

In [ ]:
df.shape

In [ ]:
genre_encoded

In [ ]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []

    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)

    return remove_whitespace_df

# This is the final one mentioned above
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()

    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df